# Pandas profiling



_This notebook was contributed by [Felix van der Spek](https://www.linkedin.com/in/felix-van-der-spek-869715140/), JADS Data Science Foundation for Experts class, November 2021_

For those seeking to improve their exploratory data analysis workflow, the [pandas profiling](https://pandas-profiling.github.io/pandas-profiling/docs/master/rtd/) is a good library to use. We demonstrate it here using the Ames housing case. Also, the metadata from the original journal paper is used to generate a properly documented report.

Since the reports can get quite large, it is recommended to save it in a separate file.

In [6]:
import json
import re
from fuzzywuzzy import fuzz
import pandas as pd
import requests
from pandas_profiling import ProfileReport


DATASET_URL = "https://github.com/jads-nl/discover-projects/blob/main/ames-housing/AmesHousing.csv?raw=true"
DATASET_METADATA_URL = "http://jse.amstat.org/v19n3/decock/DataDocumentation.txt"
VARIABLE_DEFINITION_FILENAME = "dataset_variable_definition.json"

In [7]:
def fetch_variable_definitions(
    dataset_url=DATASET_URL, 
    dataset_metadata_url=DATASET_METADATA_URL, 
    filename=VARIABLE_DEFINITION_FILENAME
):
    """Fetch variable definitions from dataset metadata."""

    response = requests.get(dataset_metadata_url)

    pattern = re.compile(r'(?P<variable>.*?)(?:\s*\(.*\)\s*:\s*)(?P<description>.*)')
    matches = re.finditer(pattern, response.text)
    
    definitions = {
        match.group('variable'): match.group('description') 
        for match in matches 
        if match is not None
    }
    definitions = {k.strip(): v.strip() for k, v in definitions.items()}

    print(f"number of variables in metadata: {len(definitions)}")

    # get dataset so we can map variable names from dataset to metadata
    df = pd.read_csv(dataset_url)
    print(f"number of variables in dataset: {len(df.columns)}")

    # quick sanity check
    assert len(definitions) == len(df.columns), "number of columns don't match"

    column_names = pd.Series(df.columns)
    for variable in definitions.copy():
        # use Levenshtein distance to fuzzy match variable names from dataset to metadata
        levenshtein_scores = column_names.apply(fuzz.ratio, args=(variable,))
        column_name = column_names[levenshtein_scores == levenshtein_scores.max()].iat[0]
        
        # using the fuzz.ratio method, we incorrectly missname one variable
        # which we have to correct manually
        if variable == "Kitchen":
            column_name = "Kitchen AbvGr"
            
        # user feedback
        if variable != column_name:
            print(f"altering: {variable} -> {column_name}")

        definitions[column_name] = definitions.pop(variable)

    with open(filename, "w") as output_file:
        json.dump(obj=definitions, fp=output_file, indent=4)
        print(f"saved variable names and descriptions to {filename}")

In [8]:
fetch_variable_definitions()

number of variables in metadata: 82
number of variables in dataset: 82
altering: Exterior 1 -> Exterior 1st
altering: Exterior 2 -> Exterior 2nd
altering: BsmtFinType 2 -> BsmtFin Type 2
altering: HeatingQC -> Heating QC
altering: Bedroom -> Bedroom AbvGr
altering: Kitchen -> Kitchen AbvGr
altering: KitchenQual -> Kitchen Qual
altering: TotRmsAbvGrd -> TotRms AbvGrd
altering: FireplaceQu -> Fireplace Qu
altering: 3-Ssn Porch -> 3Ssn Porch
saved variable names and descriptions to dataset_variable_definition.json


Creating a profile report interactively in you notebook using `df.profile_report()` or `profile.to_notebook_iframe()` may cause issues since the cell output can quickly get to large. Instead, we generate a separate file with annotated variables.

In [9]:
with open(VARIABLE_DEFINITION_FILENAME) as f:
    variables = json.load(f)

In [10]:
df = pd.read_csv(DATASET_URL)
profile = ProfileReport(
    df,
    explorative=True,
    title="Ames Housing Data Set Profile",
    dataset={
        "description": "Profiling report for the Ames housing dataset.",
        "creator": "Felix van der Spek",
        "url": DATASET_URL,
    },
    variables=variables,
)
profile.to_file("ames.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/home/dkapitan/.local/lib/python3.9/site-packages/pandas_profiling/model/correlations.py:55: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'No data; `observed` has size 0.')
  warnings.warn(
/home/dkapitan/.local/lib/python3.9/site-packages/missingno/missingno.py:250: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right', fontsize=fontsize)
/home/dkapitan/.local/lib/python3.9/site-packages/pandas_profiling/model/missing.py:89: UserWarning: There was an attempt to generate the Count missing values diagrams, but this failed.
To hide this warning, disable the calculation
(using `d

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Download and open the file in Google Chrome or Firefox. Wait for the file to load completely, after which navigation will work.